# MLOps Final Project:

Comparing performance of fine-tuned LLM vs out of the box pretrained LLM on sentiment analysis task for IMDB reviews dataset

# Data Preprocessing

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

# Load the dataset
df = pd.read_csv('movie_short.csv')

# Split into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Prepare the datasets for training
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/opt/homebrew/anaconda3/envs/DistributedComputing/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/DistributedComputing/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 200/200 [00:00<00:00, 561.29 examples/s]


# DistilBERT Fine-Tuning

In [2]:
# Load the pre-trained model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 10/300 [00:06<02:43,  1.77it/s]

{'loss': 0.691, 'grad_norm': 1.2544898986816406, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.1}


  7%|▋         | 20/300 [00:11<02:32,  1.84it/s]

{'loss': 0.7051, 'grad_norm': 1.4241828918457031, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.2}


 10%|█         | 30/300 [00:16<02:27,  1.83it/s]

...

 97%|█████████▋| 290/300 [02:37<00:05,  1.86it/s]

{'loss': 0.2979, 'grad_norm': 0.3696131706237793, 'learning_rate': 2.9e-05, 'epoch': 2.9}


100%|██████████| 300/300 [02:42<00:00,  1.84it/s]

{'loss': 0.1945, 'grad_norm': 48.17932891845703, 'learning_rate': 3e-05, 'epoch': 3.0}
{'train_runtime': 162.6225, 'train_samples_per_second': 14.74, 'train_steps_per_second': 1.845, 'train_loss': 0.4633541278044383, 'epoch': 3.0}


TrainOutput(global_step=300, training_loss=0.4633541278044383, metrics={'train_runtime': 162.6225, 'train_samples_per_second': 14.74, 'train_steps_per_second': 1.845, 'total_flos': 317524354578432.0, 'train_loss': 0.4633541278044383, 'epoch': 3.0})

In [6]:
# from transformers import pipeline
# from sklearn.metrics import accuracy_score, f1_score

# def evaluate_model(model, tokenizer, dataset):
#     nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
#     texts = dataset['text'].tolist()  # Convert the column to a list of strings
#     preds = [1 if item['label'] == 'POSITIVE' else 0 for item in nlp(texts)]
#     labels = dataset['label'].tolist()  # Convert the column to a list of labels
#     accuracy = accuracy_score(labels, preds)
#     f1 = f1_score(labels, preds)
#     return accuracy, f1

# # Evaluate fine-tuned model
# finetuned_model = model
# finetuned_accuracy, finetuned_f1 = evaluate_model(finetuned_model, tokenizer, val_df)

# # Load the original pre-trained model (non-fine-tuned)
# pretrained_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
# pretrained_accuracy, pretrained_f1 = evaluate_model(pretrained_model, tokenizer, val_df)

# # Print the results
# print(f'Fine-tuned Model - Accuracy: {finetuned_accuracy}, F1 Score: {finetuned_f1}')
# print(f'Pre-trained Model - Accuracy: {pretrained_accuracy}, F1 Score: {pretrained_f1}')


# Evaulating DistilBERT fine-tuned vs pretrained

In [5]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm  # Use standard tqdm

def evaluate_model(model, dataset, batch_size=8):
    dataloader = DataLoader(dataset, batch_size=batch_size)
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    preds = []
    labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader):  # Use standard tqdm
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels.extend(batch['label'].cpu().numpy())
            
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return accuracy, f1

# Evaluate fine-tuned model
finetuned_model = model
finetuned_accuracy, finetuned_f1 = evaluate_model(finetuned_model, val_dataset)

# Load the original pre-trained model (non-fine-tuned)
pretrained_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
pretrained_accuracy, pretrained_f1 = evaluate_model(pretrained_model, val_dataset)

# Print the results
print(f'Fine-tuned Model - Accuracy: {finetuned_accuracy}, F1 Score: {finetuned_f1}')
print(f'Pre-trained Model - Accuracy: {pretrained_accuracy}, F1 Score: {pretrained_f1}')


100%|██████████| 25/25 [00:14<00:00,  1.73it/s]
/opt/homebrew/anaconda3/envs/DistributedComputing/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 25/25 [00:14<00:00,  1.75it/s]

Fine-tuned Model - Accuracy: 0.88, F1 Score: 0.8723404255319149
Pre-trained Model - Accuracy: 0.51, F1 Score: 0.0


# Same process as above but on another model?